### Notebook for the label transfer of PBMCs using `scANVI`

- **Developed by:** Carlos Talavera-López Ph.D
- **Würzburg Institute for Systems Immunology & Julius-Maximilian-Universität Würzburg**
- v230809

### Import required modules

In [1]:
import scvi
import anndata
import warnings
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt

/Users/cartalop/mambaforge/envs/scvi-tools/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/Users/cartalop/mambaforge/envs/scvi-tools/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/Users/cartalop/mambaforge/envs/scvi-tools/lib/python3.9/site-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/cartalop/mambaforge/envs/scvi-tools/lib/python3.9/site-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is depreca

### Set up working environment

In [2]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.8.0
scanpy      1.9.3
-----
PIL                 9.4.0
absl                NA
aiohttp             3.8.4
aiosignal           1.3.1
anyio               NA
appnope             0.1.3
asttokens           NA
async_timeout       4.0.2
attr                23.1.0
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
bs4                 4.12.2
certifi             2022.12.07
cffi                1.15.1
charset_normalizer  2.1.1
chex                0.1.6
click               8.1.3
colorama            0.4.6
contextlib2         NA
croniter            NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.6
decorator           5.1.1
deepdiff            6.3.0
docrep              0.3.2
executing           1.2.0
fastapi             0.99.1
flax                0.5.0
frozenlist          1.3.3
fsspec              2023.3.0
h5py                3.8.0
hypergeom_ufunc     NA
idna                3.4
igraph   

In [3]:
warnings.simplefilter(action = 'ignore')
scvi.settings.seed = 1712
%config InlineBackend.print_figure_kwargs = {'facecolor' : "w"}
%config InlineBackend.figure_format = 'retina'

Global seed set to 1712


In [4]:
arches_params = dict(
    use_layer_norm = "both",
    use_batch_norm = "none",
    encode_covariates = True,
    dropout_rate = 0.2,
    n_layers = 2,
)

### Read in Healthy data

In [5]:
healthy_raw = sc.read_h5ad('../data/merged_pbmc_yoshida-imyoo_QCed_pre-processed_ctl230810.h5ad')
healthy_raw

AnnData object with n_obs × n_vars = 250158 × 31908
    obs: 'sample', 'seed_labels', 'donor', 'cell_states', 'generator', 'n_genes', 'doublet_scores', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score'
    var: 'name-ImYoo_2023', 'id-ImYoo_2023', 'name-YoshidaM_2022', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'generator_colors', 'sample_colors'

### Perform joint QC analysis

#### Select a subset of cells that are unlabeled. 

In [6]:
healthy_raw.obs['seed_labels'].cat.categories

Index([' CD14+IL6+Monocyte', ' hCD4+T ', 'AS-DC', 'Ageing_B', 'Baso/Eos',
       'CD4+T', 'CD8+T', 'CD14+CD16+Monocytes', 'CD14+Monocyte',
       'CD14+Monocytes', 'CD14+Monocytes-HSP_artifact', 'CD16+C1+Monocyte',
       'CD16+Monocyte', 'CD16+Monocytes', 'CD56+NK', 'CD56_dimNK',
       'CLL-associated_B', 'HPC', 'IFN-stim_CD14+Monocyte',
       'IFN-stim_CD16+Monocyte', 'IFN-stim_HPC', 'IFN-stim_NK',
       'IFN-stim_ctlCD8+T', 'IFN-stim_n-sw_memB', 'IFN-stim_naiveCD4+T',
       'IFN-stim_naive_B', 'ILC', 'IgM_memB', 'MAIT', 'Mast', 'NK', 'NKT',
       'RBC', 'adaptive_NK', 'asDC', 'cDC1', 'cDC2', 'cDC3', 'class_memB',
       'cmCD8+T', 'ctlCD4+T  ', 'ctlCD8+T', 'cycling', 'emCD8+T', 'emraCD8+T',
       'gdT', 'invarB', 'muco_invarT', 'n-sw_memB', 'naiveCD4+T', 'naiveCD8+T',
       'naive_B', 'nan', 'pDC', 'plasma_B', 'plasmablasts', 'platelets',
       'regT', 'sw_memB', 'tumorDC'],
      dtype='object')

In [7]:
healthy_raw.obs['sample'].value_counts()

NP18    9799
NP13    8945
AN13    8244
NP23    8038
NP24    7376
        ... 
911     1154
20      1141
AN3     1041
909      924
AN1      915
Name: sample, Length: 63, dtype: int64

In [8]:
if pd.api.types.is_categorical_dtype(healthy_raw.obs['seed_labels']):
    healthy_raw.obs['seed_labels'] = healthy_raw.obs['seed_labels'].astype(str)

healthy_raw.obs['seed_labels'].replace('nan', 'Unknown', inplace = True)
healthy_raw.obs['seed_labels'] = healthy_raw.obs['seed_labels'].astype('category')

In [9]:
healthy_raw.obs['seed_labels'].cat.categories

Index([' CD14+IL6+Monocyte', ' hCD4+T ', 'AS-DC', 'Ageing_B', 'Baso/Eos',
       'CD14+CD16+Monocytes', 'CD14+Monocyte', 'CD14+Monocytes',
       'CD14+Monocytes-HSP_artifact', 'CD16+C1+Monocyte', 'CD16+Monocyte',
       'CD16+Monocytes', 'CD4+T', 'CD56+NK', 'CD56_dimNK', 'CD8+T',
       'CLL-associated_B', 'HPC', 'IFN-stim_CD14+Monocyte',
       'IFN-stim_CD16+Monocyte', 'IFN-stim_HPC', 'IFN-stim_NK',
       'IFN-stim_ctlCD8+T', 'IFN-stim_n-sw_memB', 'IFN-stim_naiveCD4+T',
       'IFN-stim_naive_B', 'ILC', 'IgM_memB', 'MAIT', 'Mast', 'NK', 'NKT',
       'RBC', 'Unknown', 'adaptive_NK', 'asDC', 'cDC1', 'cDC2', 'cDC3',
       'class_memB', 'cmCD8+T', 'ctlCD4+T  ', 'ctlCD8+T', 'cycling', 'emCD8+T',
       'emraCD8+T', 'gdT', 'invarB', 'muco_invarT', 'n-sw_memB', 'naiveCD4+T',
       'naiveCD8+T', 'naive_B', 'pDC', 'plasma_B', 'plasmablasts', 'platelets',
       'regT', 'sw_memB', 'tumorDC'],
      dtype='object')

### Select highly variable genes

In [11]:
adata_raw = healthy_raw.copy()
adata = healthy_raw.copy()
adata.layers['counts'] = adata.X.copy()

sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 7000,
    layer = "counts",
    batch_key = "sample",
    subset = True
)
adata

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)


AnnData object with n_obs × n_vars = 250158 × 7000
    obs: 'sample', 'seed_labels', 'donor', 'cell_states', 'generator', 'n_genes', 'doublet_scores', 'predicted_doublets', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score'
    var: 'name-ImYoo_2023', 'id-ImYoo_2023', 'name-YoshidaM_2022', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'generator_colors', 'sample_colors', 'hvg'
    layers: 'counts'

### Transfer of annotation with scANVI

In [13]:
scvi.model.SCVI.setup_anndata(adata, 
                              batch_key = 'donor', 
                              labels_key = "seed_labels",
                              categorical_covariate_keys = ['donor', 'sample', 'generator'],
                              continuous_covariate_keys=['n_genes', 'pct_counts_mt', 'n_counts']
                              )

In [14]:
scvi_model = scvi.model.SCVI(adata, 
                             n_latent = 50, 
                             n_layers = 3, 
                             dispersion = 'gene-batch', 
                             gene_likelihood = 'nb')

In [15]:
scvi_model.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 29/32:  88%|████████▊ | 28/32 [1:02:42<08:51, 132.76s/it, v_num=1, train_loss_step=2.99e+3, train_loss_epoch=2.93e+3]

### Label transfer with `scANVI` 

In [ ]:
scanvi_model = scvi.model.SCANVI.from_scvi_model(scvi_model, 'Unknown')

In [ ]:
scanvi_model.train()

In [ ]:
adata.obs["C_scANVI"] = scanvi_model.predict(adata)

- Extract latent representation

In [ ]:
adata.obsm["X_scANVI"] = scanvi_model.get_latent_representation(adata)

- Visualise corrected dataset

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scANVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.3, spread = 4, random_state = 1712)
sc.pl.umap(adata, frameon = False, color = ['donor', 'cell_states', 'sample', 'generator', 'seed_labels'], size = 1, legend_fontsize = 5, ncols = 3)

### Export annotated object

In [ ]:
SCC0120_1_pbmc_annotated = adata[adata.obs['group'].isin(['SCC0120_1_pbmc'])]
SCC0120_1_pbmc_annotated

In [ ]:
SCC0120_1_pbmc_annotated.obs = SCC0120_1_pbmc_annotated.obs[['orig.ident', 'Age_group', 'BMI', 'COVID_severity', 'COVID_status', 'Ethnicity', 'Group', 'Sex', 'annotation_broad', 'annotation_detailed', 'sample_id', 'seed_labels', 'donor', 'cell_states', 'nCount_RNA', 'nFeature_RNA', 'nCount_HTO', 'nFeature_HTO', 'nCount_CITE', 'nFeature_CITE', 'nCount_PROT', 'nFeature_PROT', 'percent.mt', 'sample', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'tissue', 'condition', 'n_genes', 'doublet_scores', 'hashtag', 'unique', 'group', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'S_score', 'G2M_score', '_scvi_batch', '_scvi_labels', 'C_scANVI']]
SCC0120_1_pbmc_annotated